# How to compute a correspondence with functional maps

This is the first demo on how to use geomfum to compute a correspondence with functional maps. In this notebook we will see what does it mean to compute a functional map and what are the main steps to do so. Lets Start!

# Importing and visualizing shapes

First of all, we need to import our data, we will start from a pair of shapes from the FAUST dataset (https://faust-leaderboard.is.tuebingen.mpg.de/)

In [ ]:
from geomfum.shape import TriangleMesh
from geomfum.plot import MeshPlotter
from geomfum.datasets import NotebooksDataset
#load the shapes from the dataset
dataset = NotebooksDataset()
mesh_a = TriangleMesh.from_file(dataset.get_filename("tr_reg_000"))
mesh_b = TriangleMesh.from_file(dataset.get_filename("tr_reg_001"))


We can visualize our shapes with the MeshPlotter

In [ ]:
plotter = MeshPlotter.from_registry(which="plotly")
plotter.add_mesh(mesh_a)
plotter.show()
plotter.add_mesh(mesh_b)
plotter.show()

# Compute Basis
Now that we have our pair of shapes, we can compute their basis.
The basis of a shape is a set of functions that represent a basis for the space of squared integrable function defined on the surface of the shape. There are different kind of basis, but usually we consider the eigenfunctions of the Laplace beltrami operator.

In [ ]:
#compute basis
k=50 #we select the number of basis that we want to compute
mesh_a.laplacian.find_spectrum(spectrum_size=200, set_as_basis=True)
mesh_b.laplacian.find_spectrum(spectrum_size=200, set_as_basis=True)

#we can visualize the basis functions on the shapes
plotter = MeshPlotter.from_registry(colormap="RdBu", which="plotly")
plotter.add_mesh(mesh_a)
plotter.set_vertex_scalars(mesh_a.basis.vecs[:,3])
plotter.show()

plotter.add_mesh(mesh_b)
plotter.set_vertex_scalars(mesh_b.basis.vecs[:,3])
plotter.show()



# Compute Descriptors
Another set of functions that are useful in functional maps are the descriptors. While the basis describes the function space, the descriptors are functions that describe the shapes, including geometric and semantic inforations that are shared by the shapes. 

In [ ]:
from geomfum.descriptor.pipeline import (
    ArangeSubsampler,
    DescriptorPipeline,
    L2InnerNormalizer,
)
from geomfum.descriptor.spectral import HeatKernelSignature, WaveKernelSignature
import numpy as np

Landmarks: a lot of time, the informations provided by the descriptors are not enough to compute a good functional map, for this reason, a good thing is to privide some points that we know are in correspondence, these are called landmarks

In [ ]:
# select the landmarks with the plot functions, for example, considering indexes of hands, feets and head
plotter = MeshPlotter.from_registry(colormap="RdBu", which="plotly")
plotter.add_mesh(mesh_a)
plotter.show()

mesh_a.landmark_indices =np.array( [412, 5891,6593,3323,2119] )
mesh_b.landmark_indices =np.array( [412, 5891,6593,3323,2119]) #in this case the landmarks have the same indexes, but in general this is not true


In [ ]:
# now we can compute the descriptrors
steps = [
    HeatKernelSignature.from_registry(n_domain=100, use_landmarks=True),
    ArangeSubsampler(subsample_step=2),
    WaveKernelSignature.from_registry(n_domain=3),
    L2InnerNormalizer(),
]

pipeline = DescriptorPipeline(steps)

descr_a = pipeline.apply(mesh_a)
descr_b = pipeline.apply(mesh_b)

In [ ]:
# we can visualize the descriptors
plotter.add_mesh(mesh_a)
plotter.set_vertex_scalars(descr_a[0])
plotter.show()
plotter.add_mesh(mesh_b)
plotter.set_vertex_scalars(descr_b[0])
plotter.show()

# Optimize Functional Map
Now, we have all the elements to optimize our first functional maps, we can select different energies and weight them differently

In [ ]:
from geomfum.functional_map import (
    FactorSum,
    LBCommutativityEnforcing,
    OperatorCommutativityEnforcing,
    SpectralDescriptorPreservation,
)
from geomfum.numerics.optimization import ScipyMinimize


In [ ]:
#we select the number of eigenfunctions for our functional maps
mesh_a.basis.use_k=20
mesh_b.basis.use_k=20

In [ ]:


factors = [
    SpectralDescriptorPreservation(
        mesh_a.basis.project(descr_a),
        mesh_b.basis.project(descr_b),
        weight=1.0,
    ),
    LBCommutativityEnforcing.from_bases(
        mesh_a.basis,
        mesh_b.basis,
        weight=1e-2,
    ),
    OperatorCommutativityEnforcing.from_multiplication(
        mesh_a.basis, descr_a, mesh_b.basis, descr_b, weight=1e-1
    ),
    OperatorCommutativityEnforcing.from_orientation(
        mesh_a, descr_a, mesh_b, descr_b, weight=1e-1
    ),
]

objective = FactorSum(factors)

optimizer = ScipyMinimize(
    method="L-BFGS-B",
)

x0 = np.zeros((mesh_b.basis.spectrum_size, mesh_a.basis.spectrum_size))

res = optimizer.minimize(
    objective,
    x0,
    fun_jac=objective.gradient,
)

fmap = res.x.reshape(x0.shape)


In [ ]:
#we can visualize our functional map
import matplotlib.pyplot as plt

plt.imshow(fmap, cmap='bwr')

# Get the correspondence
Once we have the functional map, we can compute the point-to-point correspondence performing a nearest search in the space of function

In [ ]:
from geomfum.convert import P2pFromFmConverter

p2p_converter = P2pFromFmConverter()

p2p = p2p_converter(fmap, mesh_a.basis, mesh_b.basis)

In some cases, we can evaluate the estimated correspondence, in order ot do so we need a ground truth supervision and a distance matrix

In [ ]:
from geomfum.metric.mesh import VertexEuclideanMetric

p2p_gt=np.arange(mesh_a.n_vertices)


#compute Euclidean distance matrix
metric_a=VertexEuclideanMetric(mesh_a)
eucl_a=metric_a.dist_matrix()

print(np.mean(eucl_a[p2p,p2p_gt]))

# Refining the correspondence
A lot of time, a small number of basis is not enough to have a good correspondence and a lot of methods are based on a 'refinement' stage

In [ ]:
from geomfum.refine import ZoomOut

In [ ]:
zoomout = ZoomOut(nit=20, step=5)

zoomout_fmap_matrix_ = zoomout(fmap, mesh_a.basis, mesh_b.basis)


In [ ]:
p2p_ref=p2p_converter(zoomout_fmap_matrix_, mesh_a.basis, mesh_b.basis)
print(np.mean(eucl_a[p2p_ref,p2p_gt]))

In [ ]:
plt.imshow(zoomout_fmap_matrix_, cmap='bwr')

# How to Visualize the corrrespondence

we can visualize the quality of the correspondence

In [ ]:
plotter=MeshPlotter.from_registry(which='plotly')
plotter.colormap='viridis'

In [ ]:
colormap_a=np.mean(mesh_a.vertices,axis=-1)

plotter.add_mesh(mesh_a)
plotter.set_vertex_scalars(colormap_a)
plotter.show()

plotter.add_mesh(mesh_b)
plotter.set_vertex_scalars(colormap_a[p2p])
plotter.show()

In [ ]:
#we can also visualze the error as a function on the shape
plotter=MeshPlotter.from_registry(which='plotly',colormap='hot')
plotter.add_mesh(mesh_a)
plotter.set_vertex_scalars(eucl_a[p2p,p2p_gt])
plotter.show()

plotter.add_mesh(mesh_a)
plotter.set_vertex_scalars(eucl_a[p2p_ref,p2p_gt])
plotter.show()